## ex09-Advanced Query Techniques of CASE and Subquery

The SQLite CASE expression evaluates a list of conditions and returns an expression based on the result of the evaluation. The CASE expression is similar to the IF-THEN-ELSE statement in other programming languages. You can use the CASE statement in any clause or statement that accepts a valid expression. For example, you can use the CASE statement in clauses such as WHERE, ORDER BY, HAVING, IN, SELECT and statements such as SELECT, UPDATE, and DELETE. See more at http://www.sqlitetutorial.net/sqlite-case/. 

A subquery, simply put, is a query written as a part of a bigger statement. Think of it as a SELECT statement inside another one. The result of the inner SELECT can then be used in the outer query.

In this notebook, we put these two query techniques together to calculate seasonal runoff from year-month data in the table of rch.

In [1]:
%load_ext sql

### 1. Connet to the given database of demo.db3

In [2]:
%sql sqlite:///data/demo.db3

u'Connected: @data/demo.db3'

If you do not remember the tables in the demo data, you can always use the following command to query.

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///data/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Chek the rch table

We can find that the rch table contains time series data with year and month for each river reach. Therefore, it is natural to calculate some seasonal statistics. 

In [4]:
%sql SELECT * From rch LIMIT 3

 * sqlite:///data/demo.db3
Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,ORGN_INkg,ORGN_OUTkg,ORGP_INkg,ORGP_OUTkg,NO3_INkg,NO3_OUTkg,NH4_INkg,NH4_OUTkg,NO2_INkg,NO2_OUTkg,MINP_INkg,MINP_OUTkg,CHLA_INkg,CHLA_OUTkg,CBOD_INkg,CBOD_OUTkg,DISOX_INkg,DISOX_OUTkg,SOLPST_INmg,SOLPST_OUTmg,SORPST_INmg,SORPST_OUTmg,REACTPSTmg,VOLPSTmg,SETTLPSTmg,RESUSP_PSTmg,DIFFUSEPSTmg,REACBEDPSTmg,BURYPSTmg,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
1,1981,1,146.343765259,146.252487183,0.0912808850408,0.0,2.33204616507e-07,61619.4648438,155.371902466,0.0160862877965,0.0,0.0482588782907,0.0,362.048675537,361.81350708,203.620849609,421.183776855,0.0,23.0184326172,0.0161072444171,0.0,1.18390523079e-11,0.0,0.0,0.0,5627225.0,5623486.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,806.01574707,0.0,0.0,0.0
2,1981,1,96.225692749,96.1828536987,0.0428212843835,0.0,1.64267646596e-07,0.0,0.0,0.0136315366253,0.0,0.0408946201205,0.0,315.600524902,315.457977295,0.0,127.005020142,0.0,0.0,0.0136560499668,0.0,4.13697217716e-16,0.0,0.0,0.0,3757606.5,3698301.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,442.463012695,0.0,0.0,0.0
3,1981,1,11.9527187347,11.8613681793,0.0913518294692,0.0,2.03258238685e-07,2.03258238685e-07,6.59506094181e-09,0.0114662880078,0.0,0.0343988612294,0.00911803822964,48.2963752747,47.9315032959,0.0,62.4676208496,0.0,0.0,0.0114851053804,0.0,5.9410287833e-14,0.0,0.0,0.0,360979.90625,456115.90625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.399124146,0.00911803822964,0.0,0.0


### 3. Calculate Seasonal Runoff

There are two key steps: 
>(1) use the CASE and Subquery to convert months to named seasons;<br>
>(2) calculate seasonal mean with aggregate functions on groups.

In addition, we also use another filter keyword of ***BETWEEN*** to span months into seasons.

In [5]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM(
SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch)
GROUP BY RCH, Quarter

Done.


RCH,Quarter,Runoff
1,DJF,99.2049905353
1,JJA,1405.26229799
1,MAM,559.746932019
1,SON,454.737985314
2,DJF,56.3285390854
2,JJA,773.664279811
2,MAM,203.242827762
2,SON,297.934326341
3,DJF,32.8383281098
3,JJA,166.310382387


### Summary

Sometimes, we may need construct complicated requires that go beyond a table join or basic SELECT query. For example, we might need to write a query that uses the results of other queries as inputs (i.e., SUBQUERY). Or we might need to reclassify numerical values into categories before counting them (i.e., CASE). 

In this notebook, we explored a collection of SQL functions and options essential for solving more complex problems. Now we can add subqueries in multiple locations to provide finer control over filtering or preprocessing data before analyzing it in a main query.